In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from dotenv import load_dotenv
import os
import pandas as pd
import config
import time

load_dotenv()

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
SPOTIPY_REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI')


auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

## Collect all songs released by the top artists
After collecting data for all artists, Spotipy was used to collect all Spotify songs for each artist. The following function collects data on all songs released by a given artist. Note that the maximum number of records in a specific query is 1000.

In [6]:
def get_songs(sp, artist):
    counter = 0
    
    # empty list to store data
    song_id = []
    song_name = []
    artist_id = []
    artist_name = []
    release_date = []
    song_popularity = []

    # keep executing api calls until the offset parameter reaches 1000
    while counter < 1000:
        # call api

        results = sp.search(q = f"artist:{artist}", type = "track", limit = 50, offset = 0)
        for idx, song in enumerate(results['tracks']['items']):
            try:
                if song['artists'][0]['name'] == artist:
                    song_id.append(song['id'])
                    song_name.append(song['name'])
                    artist_id.append(song['artists'][0]['id'])
                    artist_name.append(song['artists'][0]['name'])
                    release_date.append(song['album']['release_date'])
                    song_popularity.append(song['popularity'])
            except Exception as e:
                print(e)
        
        counter += 50
    
    songs_df = pd.DataFrame()
    songs_df['song_id'] = song_id
    songs_df['song_name'] = song_name
    songs_df['artist_id'] = artist_id
    songs_df['artist_name'] = artist_name
    songs_df['release_date'] = release_date
    songs_df['song_popularity'] = song_popularity
    
    return songs_df

In [ ]:
# print(type(sp))
artists = pd.read_excel('./data/artists.xlsx')
artists = list(artists['artist'])
# print(artists)
df = pd.DataFrame()

for idx, artist in enumerate(artists):
    df_artist = get_songs(sp, artist)
        
    if idx == 0:
        df = df_artist
    else:
        df = pd.concat([df, df_artist])
    time.sleep(3)
df.to_excel('./data/songs.xlsx', index = False)
df